# Webscraping contestants from ECI

## Problem Statement:
Scrape details of women contestants from the ECI website of the 2021 State Assembly elections. 

In [1]:
# Loading required packages
import pandas as pd
import numpy as np
import math


# Packages for webscraping
import os, random, sys, time
from urllib.parse import urlparse
from selenium import webdriver
from bs4 import BeautifulSoup
import requests

We'll now write the following blocks and then assemble them together:

- get_candidate_links: This will extract all candidate links from a given webpage. 

- get_details: This function will extract all the required details about a candidate from a candidate posting and append it to the dataframe.

In [50]:
base_url = 'https://affidavit.eci.gov.in/?page='
pages = [base_url+str(num) for num in range(1,1561)]

In [104]:
#Get candidate links
candidate_links = []
for page in pages:
    #if int(page.split('=')[1])%10 == 0:
     #   print(page.split('=')[1])
    req1 = requests.get(page)
    soup1 = BeautifulSoup(req1.text, "html.parser")
    for i in range(0,10):
        candidate_links.append(soup1.find_all(class_='img-bx')[i].find("a", href=True)['href'])

In [105]:
candidate_links[:5]

['https://affidavit.eci.gov.in/show-profile/MTY4OTY=/MTY=/MTA=/Mw==/QUM=',
 'https://affidavit.eci.gov.in/show-profile/MTY4OTc=/MTY=/MTA=/Mw==/QUM=',
 'https://affidavit.eci.gov.in/show-profile/MTY4OTg=/MTY=/MTA=/Mw==/QUM=',
 'https://affidavit.eci.gov.in/show-profile/MTY4OTk=/MTY=/MTA=/Mw==/QUM=',
 'https://affidavit.eci.gov.in/show-profile/MTY5MDA=/MTY=/MTA=/Mw==/QUM=']

In [106]:
len(candidate_links)

15600

In [90]:
def get_details(candidate_links,df):
    for (idx,link) in enumerate(candidate_links):
        if idx%10 == 0:
            print(idx)
        req = requests.get(link)
        soup = BeautifulSoup(req.text, "html.parser")
        
        
        details1 = soup.find_all(class_='col-sm-6')
        party = details1[1].get_text().split('\n')[4].strip()
        constituency = details1[1].get_text().split('\n')[26]
        state = details1[1].get_text().split('\n')[33]
        status = details1[1].get_text().split('\n')[-4]
        
        details = soup.find_all(class_='form-group')
        candidate_name = details[2].get_text().split('\n')[-2]
        age = details[-1].get_text().split('\n')[-2]
        gender = details[-2].get_text().split('\n')[-2]
        
        df = df.append({'Candidate':candidate_name, "Party":party, "Status":status,
                    'State':state, "Constituency":constituency, "Age":age,
                   "Gender":gender},ignore_index=True)
        
    return df

In [95]:
empty_df = pd.DataFrame(columns=["Candidate","Party","Status","State","Constituency","Age","Gender"])
empty_df

,Candidate,Party,Status,State,Constituency,Age,Gender


In [ ]:
df = get_details(candidate_links,empty_df)

In [98]:
df#.to_csv('AllCandidates')

,Candidate,Party,Status,State,Constituency,Age,Gender
0,ABDUL ALIM,Independent,Accepted,West Bengal,Manikchak,30,male
1,SWAPAN DAS,Bahujan Mukti Party,Accepted,West Bengal,Rashbehari,57,male
2,Nani Gopal Dawn,Independent,Accepted,West Bengal,Asansol Uttar,56,male
3,ALAM MOTTAKIN,Indian National Congress,Accepted,West Bengal,Manikchak,42,male
4,PREETAM KUNDU,Independent,Accepted,West Bengal,Rashbehari,41,male
...,...,...,...,...,...,...,...
15595,DIPEN BAURI,SOCIALIST UNITY CENTRE OF INDIA (COMMUNIST),Accepted,West Bengal,Saltora,49,male
15596,SURJYENDU BIKASH PATRA,SOCIALIST UNITY CENTRE OF INDIA (COMMUNIST),Accepted,West Bengal,Patashpur,36,male
15597,JAGADIS SAU,SOCIALIST UNITY CENTRE OF INDIA (COMMUNIST),Accepted,West Bengal,Egra,60,male
15598,Somnath Mandal,SOCIALIST UNITY CENTRE OF INDIA (COMMUNIST),Accepted,West Bengal,Khejuri,67,male
